# 🍽️ Weekly Themed Meal Planner MVP

A meal planning assistant that generates 7 themed dinners with consolidated shopping lists.

## Features
- 7 unique themed dinners (Monday-Sunday)
- 5 ingredients per recipe (excluding basics)
- Dietary preference support (vegan, vegetarian, keto, omnivore)
- Consolidated shopping list
- Email delivery
- LangSmith evaluation & monitoring

## 1. Installation & Setup

In [1]:
!pip install -q langchain langchain-openai langchain-anthropic langsmith pydantic gradio

## 2. Configuration

**Edit the values below before running the notebook.**

from google.colab import userdata
userdata.get('secretName')

In [2]:
# ============================================================
# CONFIGURATION - Edit these values
# ============================================================
from google.colab import userdata
# API Keys
OPENAI_API_KEY = userdata.get('openai_key')
LANGSMITH_API_KEY = userdata.get('langsmith_key')

# Model settings
OPENAI_MODEL = "gpt-4o-mini"
TEMPERATURE = 0.7

# LangSmith settings
LANGSMITH_PROJECT = "meal-planner-mvp"

# Email settings (Gmail with App Password)
EMAIL_SENDER = "bellareadsai@gmail.com"
EMAIL_PASSWORD = "your-app-password"

# Meal generation settings
INGREDIENTS_PER_RECIPE = 5
EXCLUDED_BASICS = ["salt", "pepper", "oil", "garlic", "water"]
DAYS = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

In [3]:
import os

# Set environment variables
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["LANGSMITH_API_KEY"] = LANGSMITH_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = LANGSMITH_PROJECT

## 3. Pydantic Models for Validation

In [4]:
from pydantic import BaseModel, Field, field_validator
from typing import List, Optional
from enum import Enum


class DietaryPreference(str, Enum):
    OMNIVORE = "omnivore"
    VEGETARIAN = "vegetarian"
    VEGAN = "vegan"
    KETO = "keto"


class Ingredient(BaseModel):
    """Single ingredient with quantity."""
    name: str = Field(..., description="Name of the ingredient")
    quantity: str = Field(..., description="Quantity with unit (e.g., '2 cups', '500g')")


class Recipe(BaseModel):
    """A single themed dinner recipe."""
    day: str = Field(..., description="Day of the week")
    theme: str = Field(..., description="Cuisine theme (e.g., 'Italian', 'Mexican')")
    dish_name: str = Field(..., description="Name of the dish")
    ingredients: List[Ingredient] = Field(..., description="List of 5 ingredients")
    instructions: str = Field(..., description="Brief cooking instructions")

    @field_validator('ingredients')
    @classmethod
    def validate_ingredient_count(cls, v):
        if len(v) != INGREDIENTS_PER_RECIPE:
            raise ValueError(f"Recipe must have exactly {INGREDIENTS_PER_RECIPE} ingredients, got {len(v)}")
        return v


class ShoppingItem(BaseModel):
    """Consolidated shopping list item."""
    name: str = Field(..., description="Ingredient name")
    total_quantity: str = Field(..., description="Total quantity needed")
    used_in: List[str] = Field(..., description="List of dishes using this ingredient")


class MealPlan(BaseModel):
    """Complete weekly meal plan."""
    dietary_preference: str = Field(..., description="Selected dietary preference")
    recipes: List[Recipe] = Field(..., description="List of 7 recipes for the week")
    shopping_list: List[ShoppingItem] = Field(..., description="Consolidated shopping list")

    @field_validator('recipes')
    @classmethod
    def validate_recipe_count(cls, v):
        if len(v) != 7:
            raise ValueError(f"Meal plan must have exactly 7 recipes, got {len(v)}")
        return v

    @field_validator('recipes')
    @classmethod
    def validate_unique_themes(cls, v):
        themes = [r.theme.lower() for r in v]
        if len(themes) != len(set(themes)):
            raise ValueError("All 7 themes must be unique")
        return v

## 4. LangChain Setup & Prompts

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser


def get_llm():
    """Get the configured LLM."""
    return ChatOpenAI(model=OPENAI_MODEL, temperature=TEMPERATURE)


# Main meal plan generation prompt
MEAL_PLAN_PROMPT = ChatPromptTemplate.from_messages([
    ("system", """You are a professional meal planner. Generate a weekly meal plan with EXACTLY 7 themed dinners.

STRICT RULES - FOLLOW EXACTLY:
1. Each recipe MUST have EXACTLY 5 main ingredients (do NOT count salt, pepper, oil, garlic, or water)
2. ALL 7 THEMES MUST BE COMPLETELY DIFFERENT - no duplicates allowed! Use: Italian, Mexican, Japanese, Indian, Thai, Greek, American, Chinese, French, Korean, Mediterranean, Middle Eastern, etc.
3. All ingredients MUST be real, commonly available items
4. Follow the dietary preference strictly
5. Provide realistic quantities for 2 servings

Dietary Guidelines:
- Omnivore: Any ingredients allowed
- Vegetarian: No meat or fish, eggs and dairy OK
- Vegan: No animal products at all
- Keto: Low carb, high fat, no grains/sugar/starchy vegetables

Output valid JSON matching this exact structure:
{format_instructions}"""),
    ("human", """Create a weekly meal plan:
- Diet: {dietary_preference}
- Theme requests: {theme_requests}
- Preferences: {additional_preferences}

IMPORTANT: Each day MUST have a DIFFERENT cuisine theme. Generate 7 unique themed dinners for Monday-Sunday.""")
])


# Shopping list consolidation prompt
SHOPPING_LIST_PROMPT = ChatPromptTemplate.from_messages([
    ("system", """You are a shopping list organizer. Consolidate ingredients from multiple recipes into a single shopping list.

RULES:
1. Combine same ingredients and sum their quantities
2. List which dishes use each ingredient
3. Use standard units (cups, tablespoons, grams, pieces)
4. Round up quantities for convenience

Output valid JSON matching this structure:
{format_instructions}"""),
    ("human", """Consolidate the shopping list from these recipes:
{recipes_json}""")
])

## 5. Meal Generation Logic

In [ ]:
import json
import time
from langsmith import traceable


class MealPlanGenerator:
    """Generates weekly themed meal plans."""

    def __init__(self):
        self.llm = get_llm()
        self.recipe_parser = JsonOutputParser(pydantic_object=MealPlan)
        self.shopping_parser = JsonOutputParser(pydantic_object=ShoppingItem)

    @traceable(name="generate_meal_plan")
    def generate(
        self,
        dietary_preference: str,
        theme_requests: str = "random",
        additional_preferences: str = "none"
    ) -> dict:
        """Generate a complete meal plan with shopping list."""
        start_time = time.time()

        recipe_chain = MEAL_PLAN_PROMPT | self.llm

        result = recipe_chain.invoke({
            "dietary_preference": dietary_preference,
            "theme_requests": theme_requests if theme_requests else "random diverse cuisines",
            "additional_preferences": additional_preferences if additional_preferences else "none",
            "format_instructions": self.recipe_parser.get_format_instructions()
        })

        try:
            content = result.content
            if "```json" in content:
                content = content.split("```json")[1].split("```")[0]
            elif "```" in content:
                content = content.split("```")[1].split("```")[0]

            meal_plan_data = json.loads(content)
            meal_plan = MealPlan(**meal_plan_data)
            generation_time = time.time() - start_time

            return {
                "success": True,
                "meal_plan": meal_plan.model_dump(),
                "generation_time": round(generation_time, 2),
                "error": None
            }

        except json.JSONDecodeError as e:
            return {"success": False, "meal_plan": None, "error": f"JSON parsing error: {str(e)}"}
        except Exception as e:
            return {"success": False, "meal_plan": None, "error": str(e)}


# Initialize generator
generator = MealPlanGenerator()

## 6. Email Functionality

In [ ]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart


def format_meal_plan_email(meal_plan: dict) -> str:
    """Format meal plan as readable email content (shopping list first)."""
    lines = []
    lines.append("=" * 40)
    lines.append(f"WEEKLY MEAL PLAN ({meal_plan['dietary_preference'].upper()})")
    lines.append("=" * 40)
    lines.append("")

    # Shopping list FIRST
    lines.append("SHOPPING LIST")
    lines.append("-" * 20)
    for item in meal_plan['shopping_list']:
        lines.append(f"- {item['total_quantity']} {item['name']}")
    lines.append("")

    # Recipes
    lines.append("=" * 40)
    lines.append("RECIPES")
    lines.append("=" * 40)
    for recipe in meal_plan['recipes']:
        lines.append(f"\n{recipe['day'].upper()} - {recipe['theme']}")
        lines.append(f"{recipe['dish_name']}")
        ingredients = ", ".join([f"{ing['quantity']} {ing['name']}" for ing in recipe['ingredients']])
        lines.append(f"Ingredients: {ingredients}")
        lines.append(f"Instructions: {recipe['instructions']}")

    return "\n".join(lines)


def send_meal_plan_email(recipient_email: str, meal_plan: dict) -> dict:
    """Send meal plan via email."""
    try:
        msg = MIMEMultipart()
        msg['From'] = EMAIL_SENDER
        msg['To'] = recipient_email
        msg['Subject'] = "Your Weekly Meal Plan"

        body = format_meal_plan_email(meal_plan)
        msg.attach(MIMEText(body, 'plain'))

        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login(EMAIL_SENDER, EMAIL_PASSWORD)
            server.send_message(msg)

        return {"success": True, "message": f"Email sent to {recipient_email}"}
    except Exception as e:
        return {"success": False, "message": f"Failed to send email: {str(e)}"}

## 7. LangSmith Evaluations

Custom evaluators for:
- Shopping list completeness
- Theme uniqueness and 5-ingredient rule
- No hallucinated ingredients

In [ ]:
from langsmith import Client
from langsmith.evaluation import evaluate
import json
import re

# Initialize LangSmith client
ls_client = Client()

# ============================================================
# LLM-AS-JUDGE EVALUATORS FOR LANGSMITH
# ============================================================

def parse_llm_json(content: str) -> dict:
    """Extract JSON from LLM response, handling various formats."""
    # Try direct parse first
    try:
        return json.loads(content.strip())
    except:
        pass
    
    # Remove markdown code blocks
    content = re.sub(r'```json\s*', '', content)
    content = re.sub(r'```\s*', '', content)
    
    # Try to find JSON object
    match = re.search(r'\{[^{}]*"score"[^{}]*\}', content)
    if match:
        try:
            return json.loads(match.group())
        except:
            pass
    
    # Fallback: look for score pattern
    if '"score": 1' in content or '"score":1' in content:
        return {"score": 1, "reason": "Passed"}
    return {"score": 0, "reason": "Could not parse evaluation"}


def shopping_list_completeness(run, example) -> dict:
    """LLM-as-judge: Check if shopping list contains all recipe ingredients."""
    output = run.outputs
    if not output or not output.get("success"):
        return {"key": "shopping_list_completeness", "score": 0, "comment": "Generation failed"}

    meal_plan = output["meal_plan"]
    
    # Extract all ingredients from recipes
    recipe_ingredients = []
    for recipe in meal_plan["recipes"]:
        for ing in recipe["ingredients"]:
            recipe_ingredients.append(ing["name"])
    
    shopping_items = [item["name"] for item in meal_plan["shopping_list"]]

    eval_llm = get_llm()
    prompt = f"""Check if this shopping list contains all the recipe ingredients.

RECIPE INGREDIENTS: {recipe_ingredients}

SHOPPING LIST: {shopping_items}

Does the shopping list include every ingredient? 
Respond with JSON only: {{"score": 1, "reason": "..."}} if complete, {{"score": 0, "reason": "missing: X, Y"}} if not."""

    response = eval_llm.invoke(prompt)
    result = parse_llm_json(response.content)
    return {"key": "shopping_list_completeness", "score": result.get("score", 0), "comment": result.get("reason", "")}


def theme_uniqueness(run, example) -> dict:
    """LLM-as-judge: Check if all 7 themes are unique and each has 5 ingredients."""
    output = run.outputs
    if not output or not output.get("success"):
        return {"key": "theme_uniqueness", "score": 0, "comment": "Generation failed"}

    meal_plan = output["meal_plan"]
    
    # Extract themes and ingredient counts
    themes = [r["theme"] for r in meal_plan["recipes"]]
    ingredient_counts = [len(r["ingredients"]) for r in meal_plan["recipes"]]

    eval_llm = get_llm()
    prompt = f"""Evaluate this meal plan structure.

THEMES (should all be different): {themes}
INGREDIENT COUNTS (should all be 5): {ingredient_counts}

Check: Are all 7 themes unique cuisines? Does each recipe have exactly 5 ingredients?
Respond with JSON only: {{"score": 1, "reason": "..."}} if valid, {{"score": 0, "reason": "..."}} if not."""

    response = eval_llm.invoke(prompt)
    result = parse_llm_json(response.content)
    return {"key": "theme_uniqueness", "score": result.get("score", 0), "comment": result.get("reason", "")}


def dietary_compliance(run, example) -> dict:
    """LLM-as-judge: Check for hallucinated ingredients and dietary violations."""
    output = run.outputs
    if not output or not output.get("success"):
        return {"key": "dietary_compliance", "score": 0, "comment": "Generation failed"}

    meal_plan = output["meal_plan"]
    dietary_pref = example.inputs.get("dietary_preference", "omnivore")
    
    # Extract all ingredients
    all_ingredients = []
    for recipe in meal_plan["recipes"]:
        for ing in recipe["ingredients"]:
            all_ingredients.append(ing["name"])

    eval_llm = get_llm()
    prompt = f"""Check if these ingredients comply with a {dietary_pref} diet.

DIET: {dietary_pref}
- Vegan: No animal products (meat, fish, dairy, eggs, honey)
- Vegetarian: No meat or fish
- Keto: No grains, sugar, starchy vegetables, most fruits
- Omnivore: Anything allowed

INGREDIENTS: {all_ingredients}

Are all ingredients real foods that comply with the {dietary_pref} diet?
Respond with JSON only: {{"score": 1, "reason": "all compliant"}} or {{"score": 0, "reason": "violation: X"}}"""

    response = eval_llm.invoke(prompt)
    result = parse_llm_json(response.content)
    return {"key": "dietary_compliance", "score": result.get("score", 0), "comment": result.get("reason", "")}

In [ ]:
def run_basic_validation(meal_plan_result: dict) -> dict:
    """Run quick programmatic validation checks (not LLM-based)."""
    if not meal_plan_result.get("success"):
        return {"passed": False, "errors": [meal_plan_result.get("error")]}

    meal_plan = meal_plan_result["meal_plan"]
    errors = []

    # Check 1: 7 recipes
    if len(meal_plan["recipes"]) != 7:
        errors.append(f"Expected 7 recipes, got {len(meal_plan['recipes'])}")

    # Check 2: Unique themes
    themes = [r["theme"].lower() for r in meal_plan["recipes"]]
    if len(themes) != len(set(themes)):
        duplicates = [t for t in themes if themes.count(t) > 1]
        errors.append(f"Duplicate themes: {set(duplicates)}")

    # Check 3: 5 ingredients per recipe
    for recipe in meal_plan["recipes"]:
        if len(recipe["ingredients"]) != 5:
            errors.append(f"{recipe['day']}: {len(recipe['ingredients'])} ingredients (expected 5)")

    return {"passed": len(errors) == 0, "errors": errors}

In [ ]:
# ============================================================
# TEST DATASET & LANGSMITH EVALUATION
# ============================================================

TEST_CASES = [
    {"dietary_preference": "omnivore", "theme_requests": "random", "additional_preferences": "none"},
    {"dietary_preference": "vegetarian", "theme_requests": "Italian, Asian, Mexican", "additional_preferences": "no mushrooms"},
    {"dietary_preference": "vegan", "theme_requests": "random", "additional_preferences": "high protein"},
    {"dietary_preference": "keto", "theme_requests": "Mediterranean, American", "additional_preferences": "none"}
]


def create_or_get_dataset(dataset_name: str = "meal-planner-eval"):
    """Create evaluation dataset in LangSmith if it doesn't exist."""
    try:
        dataset = ls_client.read_dataset(dataset_name=dataset_name)
        print(f"Using existing dataset: {dataset_name}")
    except:
        dataset = ls_client.create_dataset(dataset_name=dataset_name, description="Meal planner test cases")
        for test_case in TEST_CASES:
            ls_client.create_example(inputs=test_case, dataset_id=dataset.id)
        print(f"Created dataset: {dataset_name} with {len(TEST_CASES)} examples")
    return dataset


def target_function(inputs: dict) -> dict:
    """Target function for LangSmith evaluation."""
    return generator.generate(
        dietary_preference=inputs["dietary_preference"],
        theme_requests=inputs.get("theme_requests", "random"),
        additional_preferences=inputs.get("additional_preferences", "none")
    )


def run_langsmith_evaluation():
    """Run LLM-as-judge evaluation in LangSmith."""
    dataset = create_or_get_dataset()

    results = evaluate(
        target_function,
        data=dataset,
        evaluators=[
            shopping_list_completeness,
            theme_uniqueness,
            dietary_compliance
        ],
        experiment_prefix="meal-planner-llm-judge"
    )

    print("\nEvaluation complete! View results in LangSmith dashboard.")
    return results

## 8. Gradio Interface

In [ ]:
import gradio as gr
import time as pytime


def format_shopping_list(meal_plan: dict) -> str:
    """Format shopping list section."""
    lines = ["## Shopping List\n"]
    for item in meal_plan['shopping_list']:
        lines.append(f"- {item['total_quantity']} **{item['name']}**")
    return "\n".join(lines)


def format_single_recipe(recipe: dict) -> str:
    """Format a single recipe compactly."""
    ingredients = ", ".join([f"{ing['quantity']} {ing['name']}" for ing in recipe['ingredients']])
    return f"**{recipe['day']}** | {recipe['theme']} | *{recipe['dish_name']}*\n{ingredients}\n"


def generate_plan_streaming(dietary_pref, theme_requests, additional_prefs):
    """Generate meal plan with streaming output."""
    output = "Generating your meal plan..."
    yield output

    result = generator.generate(
        dietary_preference=dietary_pref,
        theme_requests=theme_requests if theme_requests.strip() else "random",
        additional_preferences=additional_prefs if additional_prefs.strip() else "none"
    )

    if not result["success"]:
        yield f"Error: {result['error']}"
        return

    meal_plan = result["meal_plan"]

    # Show shopping list first
    output = f"## Weekly {meal_plan['dietary_preference'].title()} Meal Plan\n\n"
    output += format_shopping_list(meal_plan) + "\n\n---\n\n"
    output += "## Recipes\n\n"
    yield output
    pytime.sleep(0.1)

    # Stream recipes one by one
    for recipe in meal_plan['recipes']:
        output += format_single_recipe(recipe) + "\n"
        yield output
        pytime.sleep(0.05)

    # Validation status
    validation = run_basic_validation(result)
    status = "All checks passed" if validation["passed"] else f"Issues: {', '.join(validation['errors'])}"
    output += f"---\n*Generated in {result['generation_time']}s | {status}*"
    yield output


def send_email_ui(email, dietary_pref, theme_requests, additional_prefs):
    """Generate and send meal plan via email."""
    if not email or "@" not in email:
        return "Enter a valid email address."

    result = generator.generate(
        dietary_preference=dietary_pref,
        theme_requests=theme_requests if theme_requests.strip() else "random",
        additional_preferences=additional_prefs if additional_prefs.strip() else "none"
    )

    if not result["success"]:
        return f"Generation failed: {result['error']}"

    email_result = send_meal_plan_email(email, result["meal_plan"])
    return email_result["message"]


# Create Gradio interface
with gr.Blocks(title="Weekly Meal Planner") as app:
    gr.Markdown("# Weekly Meal Planner\nGenerate 7 themed dinners with a consolidated shopping list.")

    with gr.Row():
        with gr.Column(scale=1):
            dietary_dropdown = gr.Dropdown(
                choices=["omnivore", "vegetarian", "vegan", "keto"],
                value="omnivore",
                label="Diet"
            )
            theme_input = gr.Textbox(label="Themes (optional)", placeholder="Italian, Mexican...", lines=1)
            additional_input = gr.Textbox(label="Preferences (optional)", placeholder="no nuts, quick meals...", lines=1)
            generate_btn = gr.Button("Generate", variant="primary")

            gr.Markdown("---")
            email_input = gr.Textbox(label="Email", placeholder="you@example.com")
            send_btn = gr.Button("Email Plan")
            email_status = gr.Textbox(label="Status", interactive=False)

        with gr.Column(scale=2):
            output_display = gr.Markdown("Click **Generate** to start.")

    generate_btn.click(
        fn=generate_plan_streaming,
        inputs=[dietary_dropdown, theme_input, additional_input],
        outputs=[output_display]
    )

    send_btn.click(
        fn=send_email_ui,
        inputs=[email_input, dietary_dropdown, theme_input, additional_input],
        outputs=[email_status]
    )

## 9. Launch Application

In [12]:
# Launch the Gradio app
# In Colab, this will create a public URL
app.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6611510821f9f81199.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## 10. Run LLM-as-Judge Evaluation

Run this cell to execute evaluation with LLM-as-judge evaluators in LangSmith.
The evaluators check:
- **Shopping list completeness**: All recipe ingredients in shopping list
- **Theme uniqueness**: 7 unique themes, 5 ingredients each
- **Dietary compliance**: Real ingredients, no dietary violations

In [ ]:
# Run LLM-as-judge evaluation in LangSmith
# This will create a dataset and run all 3 evaluators on each test case
evaluation_results = run_langsmith_evaluation()